In [12]:
import pandas as pd
from num2words import num2words
from utils import create_client_engine
from config import *

In [13]:
client, client_engine = create_client_engine()

## Datacatalog for Datawarehouse

In [14]:
client.execute(
    """
    SELECT folder_name, file_name, size_in_bytes, number_of_rows FROM "datacatalog"
    WHERE number_of_rows > 0 AND size_in_bytes > 0 AND data_inserted = True
    ORDER BY number_of_rows DESC
    """
)
client.fetch_all()

[('2021-01', 'clickstream-enwiki-2021-01.tsv.gz', 609189888, 31983002),
 ('2017-11', 'clickstream-enwiki-2017-11.tsv.gz', 1207233629, 25982519),
 ('2021-01', 'clickstream-jawiki-2021-01.tsv.gz', 325695363, 6609174),
 ('2021-01', 'clickstream-dewiki-2021-01.tsv.gz', 283118417, 5838401),
 ('2021-01', 'clickstream-ruwiki-2021-01.tsv.gz', 364606675, 4956355),
 ('2017-11', 'clickstream-dewiki-2017-11.tsv.gz', 7634944, 4888888),
 ('2017-12', 'clickstream-dewiki-2017-12.tsv.gz', 209305165, 4842717),
 ('2021-01', 'clickstream-frwiki-2021-01.tsv.gz', 216488681, 4639229),
 ('2021-01', 'clickstream-eswiki-2021-01.tsv.gz', 192769729, 4203548),
 ('2021-01', 'clickstream-itwiki-2021-01.tsv.gz', 91308032, 3816020),
 ('2017-11', 'clickstream-eswiki-2017-11.tsv.gz', 136339852, 3127870),
 ('2017-11', 'clickstream-ruwiki-2017-11.tsv.gz', 154513011, 2507978),
 ('2021-01', 'clickstream-zhwiki-2021-01.tsv.gz', 91926642, 2229375),
 ('2021-01', 'clickstream-plwiki-2021-01.tsv.gz', 98246153, 2152793),
 ('2017-

### Total rows inserted into the datawarehouse

In [15]:
client.execute(
    """
    SELECT SUM(number_of_rows) FROM "datacatalog"
    WHERE number_of_rows > 0 AND size_in_bytes > 0 AND data_inserted = True
    """
)
num2words(int(client.fetch_all()[0][0]), to='ordinal')


'one hundred and twelve million, eight hundred and four thousand, two hundred and sixth'

# Q&A
In this take home project we're going to be analyzing wikipedia clickstream data found here: [link](https://dumps.wikimedia.org/other/clickstream/readme.html)

1. For the dates of 2021 January, determine the top 5 "current site" requested
and print out their counts?

In [16]:
client.execute(
    """
    SELECT resource, SUM(number_of_occurrences) as total_number_of_occurrences
    FROM "clickstream"
    WHERE date = '2021-01-01'
    GROUP BY resource
    ORDER BY total_number_of_occurrences DESC
    LIMIT 5
    """
)
client.fetch_all()

[('Main_Page', Decimal('191524141')),
 ('メインページ', Decimal('20725997')),
 ('Заглавная_страница', Decimal('16634310')),
 ('Pagina_principale', Decimal('15842122')),
 ('Hyphen-minus', Decimal('15164165'))]

2. In 2017 November, how many times was "Ferrimagnetismus" requested?

In [17]:
# How many rows it has as a resource
# client.execute(
#     """
#     SELECT COUNT(*) FROM "clickstream"
#     WHERE date = '2017-11-01' AND resource = 'Ferrimagnetismus'
#     """
# )
# client.fetch_all()

In [18]:
client.execute(
    """
    SELECT resource, SUM(number_of_occurrences) as total_number_of_occurrences 
    FROM "clickstream"
    WHERE date = '2017-11-01' AND resource = 'Ferrimagnetismus'
    GROUP BY resource
    """
)
client.fetch_all()

[('Ferrimagnetismus', Decimal('1237'))]

3. In 2017 November, how many events had "other-search" as their source?

In [19]:
client.execute(
    """
    SELECT COUNT(*) FROM "clickstream"
    WHERE date = '2017-11-01' AND referrer = 'other-search'
    """
)
client.fetch_all()

[(6493854,)]

## Issues faced
1. Had to wrangle millions of rows with limited resources both interms of time and memory.
2. Among all one file has two rows with more than 500 varying characters (exact max value15912897) which I have skipped and but logged and save to a file for further analysis.
3. Used pandas for data processing and analysis which could have been better with `Pyspark` which is a more efficient data processing and analysis ecosystem.
4. For this dataset of almost 108M rows, pandas data wrangling and dump to Postgres was good enough but more than that is beyond the scope of this take home project architecture.

## Fun facts
1. Learned and explored pandas and postgres database's capabilities to work with large datasets.
2. Learned and explored Spark's capabilities to work with large datasets.
3. Implemented a robust data pipeline with logging and error handling managed by apache airflow.
4. Implemented wrapper classes and functions to make the code more readable and maintainable.
5. Maintaining data catalog for data lake creation, basic data cleaning and populating data warehouse was a fun.    